# リポジトリクローン

In [ ]:
!cd /content && git clone 'https://github.com/verlab/accelerated_features.git'
%cd /content/accelerated_features

# モデル構築

In [2]:
from modules.model import *
from modules.interpolator import InterpolateSparse2d

xfeat_model = XFeatModel().to('cpu').eval()
xfeat_model.load_state_dict(torch.load('weights/xfeat.pt', map_location='cpu'))

interpolator_bicubic = InterpolateSparse2d('bicubic')
interpolator_nearest = InterpolateSparse2d('nearest')
interpolator_bilinear = InterpolateSparse2d('bilinear')

# ONNX変換

In [ ]:
!pip install onnx onnxruntime onnxsim

In [4]:
import torch

def convert_to_onnx(
    xfeat_model,
    interpolator_bicubic,
    interpolator_nearest,
    interpolator_bilinear,
    input_width=800,
    input_height=576,
):
    # XFeat
    xfeat_model_name = 'xfeat' + '_' + str(input_width) + 'x' + str(input_height) + '.onnx'
    xfeat_dummy_input = torch.randn(1, 3, input_height, input_width, requires_grad=True)
    torch.onnx.export(
        xfeat_model,
        xfeat_dummy_input,
        xfeat_model_name,
        verbose=True,
        input_names=['input'],
        output_names=['output'],
    )

    # Interpolator
    interp_dummy_input1 = torch.randn(1, 64, 72, 100, requires_grad=True)
    interp_dummy_input2 = torch.randn(1, 4096, 2, requires_grad=True)
    interp_dummy_height = torch.tensor([input_height], dtype=torch.int32)
    interp_dummy_width = torch.tensor([input_width], dtype=torch.int32)

    interpolator_bicubic_model_name = 'interpolator_bicubic' + '_' + str(input_width) + 'x' + str(input_height) + '.onnx'
    interpolator_nearest_model_name = 'interpolator_nearest' + '_' + str(input_width) + 'x' + str(input_height) + '.onnx'
    interpolator_bilinear_model_name = 'interpolator_bilinear' + '_' + str(input_width) + 'x' + str(input_height) + '.onnx'
    torch.onnx.export(
        interpolator_bicubic,
        (interp_dummy_input1, interp_dummy_input2, interp_dummy_height, interp_dummy_width),
        interpolator_bicubic_model_name,
        verbose=True,
        input_names=["x", "pos", "H", "W"],
        dynamic_axes={
            "x": {0: "batch_size", 1: "channel", 2: "height", 3: "width"},
            "pos": {0: "batch_size", 1: "keypoints"},
            "H": {0: "H"},
            "W": {0: "W"}
        }
    )
    torch.onnx.export(
        interpolator_nearest,
        (interp_dummy_input1, interp_dummy_input2, interp_dummy_height, interp_dummy_width),
        interpolator_nearest_model_name,
        verbose=True,
        input_names=["x", "pos", "H", "W"],
        dynamic_axes={
            "x": {0: "batch_size", 1: "channel", 2: "height", 3: "width"},
            "pos": {0: "batch_size", 1: "keypoints"},
            "H": {0: "H"},
            "W": {0: "W"}
        }
    )
    torch.onnx.export(
        interpolator_bilinear,
        (interp_dummy_input1, interp_dummy_input2, interp_dummy_height, interp_dummy_width),
        interpolator_bilinear_model_name,
        verbose=True,
        input_names=["x", "pos", "H", "W"],
        dynamic_axes={
            "x": {0: "batch_size", 1: "channel", 2: "height", 3: "width"},
            "pos": {0: "batch_size", 1: "keypoints"},
            "H": {0: "H"},
            "W": {0: "W"}
        }
    )

    !onnxsim $xfeat_model_name $xfeat_model_name
    !onnxsim $interpolator_bicubic_model_name $interpolator_bicubic_model_name
    !onnxsim $interpolator_nearest_model_name $interpolator_nearest_model_name
    !onnxsim $interpolator_bilinear_model_name $interpolator_bilinear_model_name

In [ ]:
input_shape_list = [
    [320, 224],
    [640, 480],
    [800, 576],
    [320, 160],
    [640, 352],
    [960, 512],
    [256, 256],
    [512, 512],
]

for input_shape in input_shape_list:
    convert_to_onnx(
        xfeat_model,
        interpolator_bicubic,
        interpolator_nearest,
        interpolator_bilinear,
        input_width=input_shape[0],
        input_height=input_shape[1],
    )

# まとめてZip化

In [6]:
!mkdir onnx_model

In [7]:
for input_shape in input_shape_list:
    input_width=input_shape[0]
    input_height=input_shape[1]

    model_name1 = 'xfeat' + '_' + str(input_width) + 'x' + str(input_height) + '.onnx'
    model_name2 = 'interpolator_bicubic' + '_' + str(input_width) + 'x' + str(input_height) + '.onnx'
    model_name3 = 'interpolator_nearest' + '_' + str(input_width) + 'x' + str(input_height) + '.onnx'
    model_name4 = 'interpolator_bilinear' + '_' + str(input_width) + 'x' + str(input_height) + '.onnx'

    !mv $model_name1 onnx_model/
    !mv $model_name2 onnx_model/
    !mv $model_name3 onnx_model/
    !mv $model_name4 onnx_model/

In [ ]:
!zip -r onnx_model.zip onnx_model